In [1]:
# Please upload the dataset by yourself
!unzip Orchid219.zip

Archive:  Orchid219.zip
   creating: k_fold/
   creating: k_fold/1/
  inflating: k_fold/1/train.csv      
  inflating: k_fold/1/valid.csv      
   creating: k_fold/2/
  inflating: k_fold/2/train.csv      
  inflating: k_fold/2/valid.csv      
   creating: k_fold/3/
  inflating: k_fold/3/train.csv      
  inflating: k_fold/3/valid.csv      
   creating: k_fold/4/
  inflating: k_fold/4/train.csv      
  inflating: k_fold/4/valid.csv      
   creating: k_fold/5/
  inflating: k_fold/5/train.csv      
  inflating: k_fold/5/valid.csv      
   creating: random_split/
  inflating: random_split/test.csv   
  inflating: random_split/train.csv  
  inflating: random_split/valid.csv  
  inflating: label.csv               
  inflating: submission_template.csv  
   creating: images/
  inflating: images/02a91mzn84.jpg   
  inflating: images/02nehv1tf6.jpg   
  inflating: images/02sx9ijfd6.jpg   
  inflating: images/032p7z15ol.jpg   
  inflating: images/03jezhu9i8.jpg   
  inflating: images/03ol9zqjn7.

In [12]:
# Load pretrained models
!gdown https://drive.google.com/uc?id=1rmVdM3YxQ0sOQMzc7sGgyhMU8NQQ4Lz8 
!gdown https://drive.google.com/uc?id=1aNRfXNeaABQg7kYC1TG6iZgosBMr7Pfy
!gdown https://drive.google.com/uc?id=1aokEZPfftFbxJ3G7IRrhlcLxWMo-raw5 
!gdown https://drive.google.com/uc?id=1BwJF_repxogGOpexSmtcxiWtPKx2htwr 


Downloading...
From: https://drive.google.com/uc?id=1rmVdM3YxQ0sOQMzc7sGgyhMU8NQQ4Lz8
To: /content/swin_best.pth
100% 796M/796M [00:05<00:00, 138MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1aNRfXNeaABQg7kYC1TG6iZgosBMr7Pfy
To: /content/convnext_xlarge.pth
100% 1.39G/1.39G [00:19<00:00, 72.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aokEZPfftFbxJ3G7IRrhlcLxWMo-raw5
To: /content/cnn_swin_fusion.pth
100% 2.21G/2.21G [00:20<00:00, 110MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1BwJF_repxogGOpexSmtcxiWtPKx2htwr
To: /content/beit_best.pth
100% 1.28G/1.28G [00:20<00:00, 62.1MB/s]


In [1]:
!pip install timm==0.5.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import os
import numpy as np
import random
import cv2

np.random.seed(2022)
random.seed(2022)

def train_val_split(root, data_dict, split_ratio):
    os.makedirs(os.path.join(root, "random_split"), exist_ok=True)
    train_data, valid_data, test_data = [["filename,category"]], [["filename,category"]], [["filename,category"]]

    for key, value in data_dict.items():
        label, data = key, value
        data = np.array(data)

        data_len = len(data)

        index = np.random.choice(data_len, data_len, replace=False)
        train_index = index[:int(data_len * split_ratio[0])]
        valid_index = index[int(data_len * split_ratio[0]): int(data_len * (split_ratio[0] + split_ratio[1]))]
        test_index = index[int(data_len * (split_ratio[0] + split_ratio[1])):]

        for idx in train_index:
            temp_data = [f"{data[idx]},{label}" ]
            train_data.append(temp_data)
        
        for idx in valid_index:
            temp_data = [f"{data[idx]},{label}" ]
            valid_data.append(temp_data)
        
        for idx in test_index:
            temp_data = [f"{data[idx]},{label}" ]
            test_data.append(temp_data)
    
    np.savetxt(os.path.join(root, "random_split", 'train.csv'),  train_data, fmt='%s', delimiter=',')
    np.savetxt(os.path.join(root, "random_split", 'valid.csv'),  valid_data, fmt='%s', delimiter=',')
    np.savetxt(os.path.join(root, "random_split", 'test.csv'),  test_data, fmt='%s', delimiter=',')
    return [len(train_data), len(valid_data), len(test_data)]

def cross_validation(root, data_dict, k_fold):

    for k in range(0, k_fold):
        os.makedirs(os.path.join(root, 'k_fold', str(k+1)), exist_ok=True)
        train_data, valid_data = [["filename,category"]], [["filename,category"]]
        for key, value in data_dict.items():
            label, data = key, value

            data_len = len(data)
            ratio = data_len // k_fold
            index = [i for i in range(data_len)]

            train_index = index[:(k * ratio)] + index[((k+1) * ratio):]
            valid_index = index[k*ratio : (k+1) * ratio]

            for idx in train_index:
                temp_data = [f"{data[idx]},{label}" ]
                train_data.append(temp_data)
            
            for idx in valid_index:
                temp_data = [f"{data[idx]},{label}" ]
                valid_data.append(temp_data)
        
        np.savetxt(os.path.join(root, 'k_fold', str(k+1), 'train.csv'),  train_data, fmt='%s', delimiter=',')
        np.savetxt(os.path.join(root, 'k_fold', str(k+1), 'valid.csv'),  valid_data, fmt='%s', delimiter=',')

def Norm(train_path):
    # img_h, img_w = 32, 32
    img_h, img_w = 384, 384   #根据自己数据集适当调整，影响不大
    means, stdevs = [], []
    img_list = []
    
    train_path = os.path.join(train_path, 'images')
    train_paths = os.listdir(train_path)
    imgs_path_list = []
    for image_name in train_paths:
        imgs_path_list.append(os.path.join(os.path.join(train_path, image_name)))
    
    len_ = len(imgs_path_list)
    i = 0
    for path in imgs_path_list:
        img = cv2.imread(path)
        img = cv2.resize(img, (img_w, img_h))
        img = img[:, :, :, np.newaxis]
        img_list.append(img)
        i += 1
        if i % 1000 == 0:
            print(i,'/',len_)    
    
    imgs = np.concatenate(img_list, axis=3)
    imgs = imgs.astype(np.float32) / 255.
    
    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # 拉成一行
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))
    
    # BGR --> RGB ， CV读取的需要转换，PIL读取的不用转换
    means.reverse()
    stdevs.reverse()
    return means, stdevs

if __name__ == '__main__':
    root = './'
    split_ratio = [0.7,0.2,0.1] # train/valid/test
    k_fold = 5

    data_dict = {}

    infos = np.loadtxt(os.path.join(root, 'label.csv'), delimiter=',', dtype=np.str)[1:]

    for info in infos:
        file_name, label = info

        if label not in data_dict.keys():
            data_dict[label] = []
        
        data_dict[label].append(file_name)
    
    mean, std = Norm(root)

    len_data = train_val_split(root, data_dict, split_ratio)
    # cross_validation(root, data_dict, k_fold)

    print('------------Statistics------------')
    print('Number of training data: ', len_data[0])
    print('Number of validation data: ', len_data[1])
    print('Number of testing data: ', len_data[2])
    print('Mean of dataset: ', mean)
    print('Std of dataset', std)
         


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


1000 / 2190
2000 / 2190
------------Statistics------------
Number of training data:  1534
Number of validation data:  439
Number of testing data:  220
Mean of dataset:  [0.48026907, 0.4229597, 0.36704382]
Std of dataset [0.24659315, 0.2410864, 0.24884975]


In [1]:
!nvidia-smi

Fri Jun 17 17:29:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import sys
import numpy as np
import os
import tqdm, math
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data.dataset import Dataset
from PIL import Image
import pandas as pd
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.metrics import confusion_matrix
from torch import nn
from timm.data.mixup import Mixup
import timm
import torch.nn.functional as F
from torch.optim.lr_scheduler import _LRScheduler
import warnings

size = 384
train_transforms = transforms.Compose([
                                    transforms.RandomResizedCrop(size=(size,size), scale=(0.2, 1)),\
                                    transforms.RandomHorizontalFlip(0.5),   #隨機水平翻轉
                                    transforms.RandomVerticalFlip(0.5),
                                    transforms.RandomApply([transforms.ColorJitter(brightness=0.4, \
                                        contrast=0.4, saturation=0.4, hue=0.1)], p = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.480, 0.423, 0.367], [0.247, 0.241, 0.249]),
                                    transforms.RandomErasing(p = 0.25)
                                ])

test_transforms = transforms.Compose([  
                                        transforms.Resize((size,size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.480, 0.423, 0.367], [0.247, 0.241, 0.249])
                                    ])

class OrchidData(Dataset):
    def __init__(self, root, file = 0, mode= 'train'):
        if mode == 'train':
            self.transform = train_transforms
        elif mode == 'valid' or mode == 'test':
            self.transform = test_transforms  
        self.mode = mode
        self.root = root
        self.image_info = pd.read_csv(file).values

    def __getitem__(self, index):
        img_path, label = self.image_info[index]
        img = Image.open(os.path.join(self.root,img_path)).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.image_info)

def get_acc(y_preds, y_trues):
    total_correct = np.sum(np.equal(y_trues, y_preds))
    total = len(y_trues)
    accuracy = total_correct / total
    return accuracy

def get_wp_f1(y_trues, y_preds, TotalImageCount, num_classes):
    """ Precision_Recall_F1score metrics
    y_pred: the predicted score of each class, shape: (Batch_size, num_classes)
    y_true: the ground truth labels, shape: (Batch_size,) for 'multi-class' or (Batch_size, n_classes) for 'multi-label'
    """
    eps=1e-20
    confusion = confusion_matrix(y_trues, y_preds)

    precision_list = []
    recall_list = []
    F1_dict = {}
    TP_list = []
    FN_list = []
    total_F1 = 0
    for i in range(len(confusion)):
        TP = confusion[i, i]
        FN = sum(confusion[i, :]) - TP
        FP = sum(confusion[:, i]) - TP

        precision = TP / (TP + FP + eps)
        recall = TP / (TP + FN + eps)
        F1 = 2 * precision  * recall / (precision + recall + eps)
        total_F1 += F1

        TP_list.append(TP)
        FN_list.append(FN)
        precision_list.append(precision)
        recall_list.append(recall)
        F1_dict[i] = F1
        

    weighted = 0.
    for i in range(len(confusion)):
        weighted += precision_list[i] * (TP_list[i] + FN_list[i])

    WP = weighted / TotalImageCount
    acc = get_acc(y_preds, y_trues)
    Macro_F1 = total_F1 / num_classes
    Final_Score = 0.5*acc + 0.5*Macro_F1
    return acc, Macro_F1, Final_Score, F1_dict

class SoftTargetCrossEntropy(nn.Module):
    def __init__(self):
        super(SoftTargetCrossEntropy, self).__init__()

    def forward(self, x, target):
        loss = torch.sum(-target * F.log_softmax(x, dim=-1), dim=-1)
        return loss.mean()

class LinearWarmupCosineAnnealingLR(_LRScheduler):
    def __init__(
        self,
        optimizer,
        warmup_epochs,
        max_epochs,
        warmup_start_lr = 0.0,
        eta_min = 0.0,
        last_epoch = -1,
    ):
        """
        Args:
            optimizer (Optimizer): Wrapped optimizer.
            warmup_epochs (int): Maximum number of iterations for linear warmup
            max_epochs (int): Maximum number of iterations
            warmup_start_lr (float): Learning rate to start the linear warmup. Default: 0.
            eta_min (float): Minimum learning rate. Default: 0.
            last_epoch (int): The index of last epoch. Default: -1.
        """
        self.warmup_epochs = warmup_epochs
        self.max_epochs = max_epochs
        self.warmup_start_lr = warmup_start_lr
        self.eta_min = eta_min

        super(LinearWarmupCosineAnnealingLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        """
        Compute learning rate using chainable form of the scheduler
        """
        if not self._get_lr_called_within_step:
            warnings.warn(
                "To get the last learning rate computed by the scheduler, "
                "please use `get_last_lr()`.",
                UserWarning,
            )

        if self.last_epoch == 0:
            return [self.warmup_start_lr] * len(self.base_lrs)
        elif self.last_epoch < self.warmup_epochs:
            return [
                group["lr"] + (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]
        elif self.last_epoch == self.warmup_epochs:
            return self.base_lrs
        elif (self.last_epoch - 1 - self.max_epochs) % (2 * (self.max_epochs - self.warmup_epochs)) == 0:
            return [
                group["lr"] + (base_lr - self.eta_min) *
                (1 - math.cos(math.pi / (self.max_epochs - self.warmup_epochs))) / 2
                for base_lr, group in zip(self.base_lrs, self.optimizer.param_groups)
            ]

        return [
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs))) /
            (
                1 +
                math.cos(math.pi * (self.last_epoch - self.warmup_epochs - 1) / (self.max_epochs - self.warmup_epochs))
            ) * (group["lr"] - self.eta_min) + self.eta_min for group in self.optimizer.param_groups
        ]

    def _get_closed_form_lr(self):
        """
        Called when epoch is passed as a param to the step function of the scheduler.
        """
        if self.last_epoch < self.warmup_epochs:
            return [
                self.warmup_start_lr + self.last_epoch * (base_lr - self.warmup_start_lr) / (self.warmup_epochs - 1)
                for base_lr in self.base_lrs
            ]

        return [
            self.eta_min + 0.5 * (base_lr - self.eta_min) *
            (1 + math.cos(math.pi * (self.last_epoch - self.warmup_epochs) / (self.max_epochs - self.warmup_epochs)))
            for base_lr in self.base_lrs
        ]

if __name__ in "__main__":
    os.environ['CUDA_VISIBLE_DEVICES'] = "0"
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print("Using device:", device)
    num_classes = 219
    epochs = 200
    batch_size = 8
    lr = 1e-4
    model_name = "BestSwin_large_patch4_window12_384_in22k_mixup_0.1.pth"
    train_data = OrchidData(root = "./images", file = "./random_split/train.csv", mode = 'train')
    valid_data = OrchidData(root = "./images", file = "./random_split/valid.csv", mode = 'valid')
    test_data = OrchidData(root = ".//images", file = "./random_split/test.csv", mode = 'test')
    trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True, num_workers = 0, drop_last=True)
    vaildloader = torch.utils.data.DataLoader(valid_data, batch_size = 1, shuffle = False, num_workers = 0, drop_last=False)
    testloader = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False, num_workers = 0, drop_last=False)

    model = timm.create_model('swin_large_patch4_window12_384_in22k', pretrained=True, num_classes=num_classes)
    '''
    'swin_large_patch4_window12_384_in22k'
    'convnext_xlarge_384_in22ft1k'
    '''
    model = model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
    scheduler = LinearWarmupCosineAnnealingLR(optimizer, warmup_epochs = 10, max_epochs = epochs, warmup_start_lr = 0.0, eta_min = 1e-6, last_epoch = -1)
    mixup_fn = Mixup(mixup_alpha=0.6, cutmix_alpha=0.4, cutmix_minmax=None, prob=0.3, switch_prob=0.5, mode='batch',label_smoothing= 0.1, num_classes = num_classes)
    criterion = SoftTargetCrossEntropy()

    Best_Valid_Score = float('-inf')
    Best_Valid_F1 = []
    Best_test_Score = float('-inf')
    Best_test_F1 = []

    for epoch in range(1, epochs):
        print("------ epoch %d/%d ------"%(epoch, epochs))
        model.train()
        for param_group in optimizer.param_groups:
            learning_rate = param_group['lr']

        running_loss = 0
        TP = np.zeros(num_classes)
        FP = np.zeros(num_classes)
        FN = np.zeros(num_classes)
        TotalImageCount = 0.

        y_preds = []
        y_trues = []
        tl = tqdm.tqdm(total=len(trainloader), ncols=0, desc="Train", unit=" step")
        for idx,(inputs, labels) in enumerate(trainloader):
            inputs, _labels = inputs.to(device), labels.to(device)
            inputs, labels = mixup_fn(inputs, _labels)
            optimizer.zero_grad()
            logps = model(inputs)
            loss = criterion(logps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            TotalImageCount += inputs.size(0)
        
            y_pred = torch.argmax(logps, dim=1)
            y_pred = y_pred.cpu().detach().numpy()
            y_true = _labels.cpu().detach().numpy()
            y_preds += list(y_pred)
            y_trues += list(y_true)

            tl.update()
            tl.set_postfix(
                lr=f"{learning_rate:.8f}",
                loss=f"{running_loss:.8f}"
            )
        tl.close()
        acc, Macro_F1, Final_Score, F1_dict = get_wp_f1(y_trues, y_preds, TotalImageCount, num_classes)
        print("[Train]  lr: %0.6f  | train loss: %4.5f | ACC: %4.5f | Macro F1: %4.5f | Final_Score: %4.5f"%(learning_rate, running_loss, acc, Macro_F1, Final_Score))
        scheduler.step()

        running_loss = 0
        model.eval()
        with torch.no_grad():
            val_loss = 0
            TP = np.zeros(num_classes)
            FP = np.zeros(num_classes)
            FN = np.zeros(num_classes)
            TotalImageCount = 0.
            y_preds = []
            y_trues = []
            
            for idx,(inputs, labels) in enumerate(vaildloader):
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model(inputs)
                TotalImageCount += inputs.size(0)

                y_pred = torch.argmax(logps, dim=1)
                y_pred = y_pred.cpu().detach().numpy()
                y_true = labels.cpu().detach().numpy()
                y_preds += list(y_pred)
                y_trues += list(y_true)
            acc, Macro_F1, Final_Score, F1_dict = get_wp_f1(y_trues, y_preds, TotalImageCount, num_classes)
            if Best_Valid_Score < Final_Score and epoch != 1:
                Best_Valid_Score = Final_Score
                Best_Valid_F1 = F1_dict
                torch.save(model.state_dict(), model_name)
                print("save model!")
            print( "[Valid]  Best Final Score: %4.5f | ACC: %4.5f | Macro F1: %4.5f | Final_Score: %4.5f"%(Best_Valid_Score, acc, Macro_F1, Final_Score) )
            

            TP = np.zeros(num_classes)
            FP = np.zeros(num_classes)
            FN = np.zeros(num_classes)
            TotalImageCount = 0.
            y_preds = []
            y_trues = []
            for idx,(inputs, labels) in enumerate(testloader):
                inputs, labels = inputs.to(device), labels.to(device)
                logps = model(inputs)
                TotalImageCount += inputs.size(0)
                
                y_pred = torch.argmax(logps, dim=1)
                y_pred = y_pred.cpu().detach().numpy()
                y_true = labels.cpu().detach().numpy()
                y_preds += list(y_pred)
                y_trues += list(y_true)
            acc, Macro_F1, Final_Score, F1_dict = get_wp_f1(y_trues, y_preds, TotalImageCount, num_classes)
            if Best_test_Score < Final_Score:
                Best_test_Score = Final_Score
                Best_test_F1 = F1_dict

            print( "[Test] Best Final Score: %4.5f | ACC: %4.5f | Macro F1: %4.5f | Final_Score: %4.5f"%(Best_test_Score, acc, Macro_F1, Final_Score) )
            


Using device: cuda


/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


------ epoch 1/200 ------


Train:   2% 3/191 [00:05<05:59,  1.91s/ step, loss=16.54862595, lr=0.00000000]

KeyboardInterrupt: ignored

In [1]:
import numpy as np
import pandas as pd
import copy, os, random, math, tqdm, time
import torch
from PIL import Image
from torch import nn
from torchvision import transforms, models
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import joblib
import timm
import warnings
warnings.filterwarnings("ignore")

size = 384
test_transforms = transforms.Compose([  
            transforms.Resize((size,size)),
            transforms.ToTensor(),
            transforms.Normalize([0.480, 0.423, 0.367], [0.247, 0.241, 0.249])
        ])
class OrchidData(Dataset):
    def __init__(self, root, file = 0, mode= 'train'):
        self.transform = test_transforms  
        self.mode = mode
        self.root = root
        self.image_info = pd.read_csv(file).values
        self.image_info = self.image_info[:10]

    def __getitem__(self, index):
        img_path, label = self.image_info[index]
        img = Image.open(os.path.join(self.root,img_path)).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.image_info)

def get_acc(y_preds, y_trues):
    total_correct = np.sum(np.equal(y_trues, y_preds))
    total = len(y_trues)
    accuracy = total_correct / total
    return accuracy

def get_wp_f1(y_trues, y_preds, TotalImageCount, num_classes):
    """ Precision_Recall_F1score metrics
    y_pred: the predicted score of each class, shape: (Batch_size, num_classes)
    y_true: the ground truth labels, shape: (Batch_size,) for 'multi-class' or (Batch_size, n_classes) for 'multi-label'
    """
    eps=1e-20
    confusion = confusion_matrix(y_trues, y_preds)

    precision_list = []
    recall_list = []
    F1_dict = {}
    TP_list = []
    FN_list = []
    total_F1 = 0
    for i in range(len(confusion)):
        TP = confusion[i, i]
        FN = sum(confusion[i, :]) - TP
        FP = sum(confusion[:, i]) - TP

        precision = TP / (TP + FP + eps)
        recall = TP / (TP + FN + eps)
        F1 = 2 * precision  * recall / (precision + recall + eps)
        total_F1 += F1

        TP_list.append(TP)
        FN_list.append(FN)
        precision_list.append(precision)
        recall_list.append(recall)
        F1_dict[i] = F1
        

    weighted = 0.
    for i in range(len(confusion)):
        weighted += precision_list[i] * (TP_list[i] + FN_list[i])

    WP = weighted / TotalImageCount
    acc = get_acc(y_preds, y_trues)
    Macro_F1 = total_F1 / num_classes
    Final_Score = 0.5*acc + 0.5*Macro_F1
    return acc, Macro_F1, Final_Score, F1_dict

if __name__ in "__main__":
    os.environ['CUDA_VISIBLE_DEVICES'] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 219
    batch_size = 2
    train_data = OrchidData(root = "./images", file = "./random_split/train.csv", mode = 'train')
    valid_data = OrchidData(root = "./images", file = "./random_split/valid.csv", mode = 'valid')
    test_data = OrchidData(root = "./images", file = "./random_split/test.csv", mode = 'test')
    trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = False, num_workers = 0, drop_last=False)
    vaildloader = torch.utils.data.DataLoader(valid_data, batch_size = batch_size, shuffle = False, num_workers = 0, drop_last=False)
    testloader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = False, num_workers = 0, drop_last=False)
    
    print("features.....")
    model = timm.create_model('convnext_xlarge_384_in22ft1k', pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load('./convnext_xlarge.pth'))
    model = model.cuda().eval()

    modelswin = timm.create_model('swin_large_patch4_window12_384_in22k', pretrained=False, num_classes=num_classes)
    modelswin.load_state_dict(torch.load('./swin_best.pth'))
    modelswin = modelswin.cuda().eval()

    with torch.no_grad():
        for loader , name in [(trainloader, 'train'), (vaildloader, 'valid'), (testloader, 'test')]:
            for idx,(inputs, labels) in enumerate(loader):
                inputs, labels = inputs.to(device), labels.to(device)

                logpSwin = modelswin.forward_features(inputs)

                logps = model.forward_features(inputs)
                logps = model.head.global_pool(logps)
                logps = model.head.norm(logps)
                logps = model.head.flatten(logps)

                logps = torch.cat((logpSwin, logps), dim = 1)


                feature = torch.cat((logps, labels.unsqueeze(1)), dim = 1)
                if idx == 0:
                    features = feature
                else:
                    features = torch.cat((features, feature), dim = 0)
            features = features.cpu().detach().numpy()
            np.savetxt(name + '.csv', features, fmt='%.18e', delimiter = ',')

    print("Train.....")
    train_data = np.loadtxt('train.csv', delimiter=',')
    train_X, train_Y = train_data[:, :-1], train_data[:, -1]
    valid_data = np.loadtxt('valid.csv', delimiter=',')
    valid_X, valid_Y = valid_data[:, :-1], valid_data[:, -1]
    test_data = np.loadtxt('test.csv', delimiter=',')
    test_X, test_Y = test_data[:, :-1], test_data[:, -1]
    train_X = np.concatenate((train_X, valid_X, test_X), axis = 0)
    train_Y = np.concatenate((train_Y, valid_Y, test_Y), axis = 0)

    Logistic = LogisticRegression(penalty = 'none', # 'l1', 'l2', 'elasticnet', 'none'
                                  solver = 'newton-cg', # 'newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'
                                  multi_class = 'auto',  # 'auto', 'ovr', 'multinomial'
                                  max_iter = 100,
                                  random_state = 0,
                                  l1_ratio = 0.5,
                                  n_jobs = -1)

    models = [(Logistic, 'Logistic')]
    for model, model_name in models:
        model.fit(train_X, train_Y)
        joblib.dump(model, model_name + ".sav")

    
    print("Valid and Test .......")
    for model, model_name in models:
        model = joblib.load(model_name + ".sav")
        print('--- %s ---'%(model_name))
        for X, Y, name in [(train_X, train_Y, "Train"), (valid_X, valid_Y, "Valid"), (test_X, test_Y, "Test")]:
            preds = model.predict(X)
            acc, Macro_F1, Final_Score, F1_dict = get_wp_f1(Y, preds, len(Y), num_classes)
            print( "[%s]  Best ACC: %4.5f | Macro F1: %4.5f | Final_Score: %4.5f"%(name, acc, Macro_F1, Final_Score) )


features.....
Train.....
Valid and Test .......
--- Logistic ---
[Train]  Best ACC: 1.00000 | Macro F1: 0.04566 | Final_Score: 0.52283
[Valid]  Best ACC: 1.00000 | Macro F1: 0.02283 | Final_Score: 0.51142
[Test]  Best ACC: 1.00000 | Macro F1: 0.04566 | Final_Score: 0.52283


In [ ]:
import numpy as np
import pandas as pd
import copy, os, random, math, tqdm
import torch
from PIL import Image
from torch import nn
from torchvision import transforms, models
from torch.utils.data.dataset import Dataset
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import timm
import warnings
warnings.filterwarnings("ignore")

test_transforms = transforms.Compose([  
                                        transforms.Resize((384,384)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.480, 0.423, 0.367], [0.247, 0.241, 0.249])
                                    ])
test640_transforms = transforms.Compose([  
                                        transforms.Resize((640,640)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.480, 0.423, 0.367], [0.247, 0.241, 0.249])
                                    ])
class OrchidData(Dataset):
    def __init__(self, root, file = 0):
        self.transform = test_transforms  
        self.transform640 = test640_transforms 
        self.root = root
        self.image_info = pd.read_csv(file).values

    def __getitem__(self, index):
        img_name, _ = self.image_info[index]
        img = Image.open(os.path.join(self.root, img_name)).convert("RGB")
        if self.transform is not None:
            img384 = self.transform(img)
            img640 = self.transform640(img)
        return img384, img640, img_name

    def __len__(self):
        return len(self.image_info)

if __name__ in "__main__":
    os.environ['CUDA_VISIBLE_DEVICES'] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    num_classes = 219
    batch_size = 48
    sub_path = "./submission_template.csv"
    sub_save_path = "./Swin+Conv_384_to_640+logistic.csv"
    sub_data = OrchidData(root = "./private_and_public", file = sub_path)
    subloader = torch.utils.data.DataLoader(sub_data, batch_size = batch_size, shuffle = False, num_workers = 4, drop_last=False)
    
    modelswin = timm.create_model('swin_large_patch4_window12_384_in22k', pretrained=False, num_classes=num_classes)
    modelswin.load_state_dict(torch.load('swin_best.pth'))
    modelswin = modelswin.cuda().eval()

    model = timm.create_model('convnext_xlarge_384_in22ft1k', pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load('convnext_xlarge.pth'))
    model = model.cuda().eval()
    
    Logistic_model = joblib.load("./Logistic.sav")

    sub = pd.read_csv(sub_path).to_dict()
    file_key = { v:k for k, v in sub['filename'].items() }

    with torch.no_grad():
        for idx,(inputs, inputs640, name) in enumerate(tqdm.tqdm(subloader)):
            inputs = inputs.to(device)
            inputs, inputs640 = inputs.to(device), inputs640.to(device)

            logpSwin,_ = modelswin.forward_features(inputs)
            logpSwin = logpSwin.mean(dim=1)

            logps,_ = model.forward_features(inputs640)
            logps = model.head.global_pool(logps)
            logps = model.head.norm(logps)
            logps = model.head.flatten(logps)
            logps = torch.cat((logpSwin, logps), dim = 1)
            logps = logps.cpu().datach().numpy()
            preds = model.predict(logps)

            for ans, name in zip(preds, name):
                sub['category'][file_key[name]] = ans
    df = pd.DataFrame.from_dict(sub)
    df.to_csv(sub_save_path, index = False, header=True)
